In [15]:
# Get emails from Gmail and parse the papers and save it as excel file
%load_ext autoreload
%autoreload 2
from connect_to_service import *
from parse_gmail_message import *
from semantic_scholar import query_tldr_all_papers

import pandas as pd
import base64
import os.path as ospath

# Override the default parameters
DATA_FOLDER = "./data/"
PAPERS_LABEL = 'Subscribe/Gscholar'
SHEET_ID = '1Z5Riim21O7Ti5hHlWzriBhn2CbJibqee4psHkonCcBw'

# Create data folder if not exists
if not ospath.exists(DATA_FOLDER):
  ospath.makedirs(DATA_FOLDER)

# Call the Gmail API
creds = get_creds(DATA_FOLDER)
service = get_gmail_service(creds)
sheet_service = get_sheets_service(creds)

# Get all the messages with labels
labels = GetLabelsId(service,'me',[PAPERS_LABEL,'UNREAD'])
messages = ListMessagesWithLabels(service,"me",labels)
print (f'Found {len(messages)} messages')

# Parse the mails
pa = PaperAggregator()

for msg in messages:
  msg_content = GetMessage(service, "me", msg['id'])
  try:
    msg_str = base64.urlsafe_b64decode(msg_content['payload']['body']['data'].encode('utf-8'))
  except KeyError:
    continue

  msg_title = ''
  for h in msg_content['payload']['headers']:
    if h['name'] == 'Subject':
      msg_title = h['value']
  parser = PapersHTMLParser(msg_title)
  parser.feed(str(msg_str))

  for paper in parser.papers:
    pa.add(paper)

total_papers = len(pa.paper_list)
# Remove previously seen papers
old_pa = PaperAggregator()
old_pa.load_excel(ospath.join(DATA_FOLDER, 'archive.xlsx'))
#old_pa.load_csv(ospath.join(DATA_FOLDER, 'archive.csv'))
for paper in old_pa.paper_list:
  pa.remove(paper)

# Sort by number of refernece mails
total_new_papers = len(pa.paper_list)
print (f'Found {total_papers} papers (New: {total_new_papers}, Duplicated: {total_papers-total_new_papers})')

old_pa.merge(pa)
df = old_pa.to_dataframe()
query_tldr_all_papers(df)

# Compare with google sheet
prepare_data = PrepareDataforUpdateSheet(sheet_service, SHEET_ID, df)

df.to_excel(ospath.join(DATA_FOLDER, 'archive.xlsx'), index=False)
#df.to_csv(ospath.join(DATA_FOLDER, 'archive.csv'), index=False)
print(f'Unread: {df.query("status == 0").shape[0]}')
number_missing = df.query("tldr == ''").shape[0]
print(f'Missing TLDR: {number_missing}')
print(f"[Sheet] Number of Updated records: {len(prepare_data['write_data'])}")
print(f"[Sheet] Number of New records: {len(prepare_data['append_data'])}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 8 messages
Found 37 papers (New: 0, Duplicated: 37)
Querying TLDR for 14 papers


100%|██████████| 14/14 [00:33<00:00,  2.36s/it]


Unread: 31
Missing TLDR: 405
[Sheet] Number of Updated records: 0
[Sheet] Number of New records: 31


In [16]:
# Update google sheet
if len(prepare_data['write_data']) > 0:
  UpdateSheet(sheet_service, SHEET_ID, prepare_data['write_data'])
if len(prepare_data['append_data']) > 0:
  AppendSheet(sheet_service, SHEET_ID, prepare_data['append_data'])

341 cells appended.


In [17]:
# Mark all as read
body = {"addLabelIds": [], "removeLabelIds": ["UNREAD","INBOX"]}
for msg in messages:
  service.users().messages().modify(userId="me", id=msg['id'], body=body).execute()